# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
!pip install nltk
!pip install plotly

In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/loredanafattorini/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/loredanafattorini/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/loredanafattorini/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
# import libraries
from sqlalchemy import create_engine
import re
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.svm import LinearSVC

import pickle
import warnings
warnings.filterwarnings('ignore') 

In [4]:
# load data from database
database_filepath = 'disaster_response.db'
engine = create_engine('sqlite:///' + database_filepath)
table_name = database_filepath.replace(".db","") + "_table"
df = pd.read_sql_table(table_name, engine)

In [5]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Create X and y 
X = df['message']
y = df.iloc[:,4:]

In [7]:
category_names = df.iloc[:,4:].columns

In [8]:
category_names

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

In [9]:
category_boolean = (df.iloc[:,4:] != 0).sum().values

In [10]:
category_boolean

array([20094,  4474,   118, 10860,  2084,  1313,   724,   471,   860,
        1672,  2923,  2314,   405,   604,   298,   875,  1194,  3446,
        1705,  1201,  1333,   532,   159,   283,   120,   309,  1151,
        7297,  2155,  2443,   282,  2455,   530,  1376,  5075])

In [11]:
df.genre.value_counts()

news      13054
direct    10766
social     2396
Name: genre, dtype: int64

### 2. Write a tokenization function to process your text data

In [12]:
def tokenize(text):
    
    url_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url,'urlplaceholder')

    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [13]:
# Build a custom transformer which extracts the starting verb of a sentence
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    # Given it is a tranformer we can return the self 
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [14]:
# Pipeline1 uses Linear Support Vector Machines Classifier
pipeline1 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf1',  MultiOutputClassifier(estimator=LinearSVC(max_iter=100000, random_state=42))) 
    ])

In [15]:
# Pipeline2 uses Linear Support Vector Machines Classifier along with a custom made transformer (StartingVerbExtractor)
pipeline2 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb_transformer', StartingVerbExtractor())
        ])),

        ('clf2', MultiOutputClassifier(estimator=LinearSVC(max_iter=100000, random_state=42)))
    ])

In [16]:
# Pipeline2 uses Random Forest Classifier
pipeline3 = Pipeline([
         ('vect', CountVectorizer(tokenizer=tokenize)),
         ('tfidf', TfidfTransformer()),
         ('clf3',  MultiOutputClassifier(RandomForestClassifier(random_state=42)))
     ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [17]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

pipeline1.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x2844f9790>)),
                ('tfidf', TfidfTransformer()),
                ('clf1',
                 MultiOutputClassifier(estimator=LinearSVC(max_iter=100000,
                                                           random_state=42)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [19]:
y_pred = pipeline1.predict(X_test)

# Training accuracy
print("The training accuracy is: ")
print(accuracy_score(y_train, pipeline1.predict(X_train))*100)
print(" ")

# Test accuracy
print("The test accuracy is: ")
print(accuracy_score(y_test, y_pred)*100)
print(" ")

# Classification report
print("Classification report")
print(classification_report(y_test, y_pred, target_names=y.columns.values))

The training accuracy is: 
71.62038449801648
 
The test accuracy is: 
29.95117485505035
 
Classification report
                        precision    recall  f1-score   support

               related       0.86      0.91      0.89      5023
               request       0.77      0.63      0.69      1118
                 offer       0.00      0.00      0.00        26
           aid_related       0.74      0.70      0.72      2752
          medical_help       0.58      0.28      0.38       505
      medical_products       0.62      0.25      0.36       346
     search_and_rescue       0.71      0.22      0.33       179
              security       0.25      0.01      0.02       112
              military       0.56      0.28      0.37       218
                 water       0.78      0.62      0.69       463
                  food       0.82      0.72      0.77       749
               shelter       0.78      0.55      0.64       591
              clothing       0.76      0.30      0.43  

In [20]:
pipeline1.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf1', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf1__estimator__C', 'clf1__estimator__class_weight', 'clf1__estimator__dual', 'clf1__estimator__fit_intercept', 'clf1__estimator__intercept_scaling', 'clf1__estimator__loss', 'clf1__estimator__max_iter', 'clf1__estimator__multi_class', 'clf1__estimator__penalty', 'clf1__estimator__random_state', 'clf1__estimator__tol', 'clf1__estimator__verbose', 'clf1__estimator', 'clf1__n_jobs'])

### 6. Improve your model
Use grid search to find better parameters. 

In [21]:
parameters = {'clf1__estimator__C': [0.1, 1, 10, 100],
             'clf1__estimator__class_weight':['balanced', None]
             }

cv = GridSearchCV(pipeline1, param_grid=parameters, n_jobs=-1)
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x2844f9790>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf1',
                                        MultiOutputClassifier(estimator=LinearSVC(max_iter=100000,
                                                                                  random_state=42)))]),
             n_jobs=-1,
             param_grid={'clf1__estimator__C': [0.1, 1, 10, 100],
                         'clf1__estimator__class_weight': ['balanced', None]})

In [22]:
print("The best hyperparameters from Grid Search are:")
print(cv.best_params_)
print("")
print("The mean accuracy of a model with these hyperparameters is:")
print(cv.best_score_)

The best hyperparameters from Grid Search are:
{'clf1__estimator__C': 1, 'clf1__estimator__class_weight': None}

The mean accuracy of a model with these hyperparameters is:
0.28486418879403974


In [23]:
best_tuned_clf = cv.best_estimator_

In [24]:
best_tuned_clf.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x2844f9790>)),
                ('tfidf', TfidfTransformer()),
                ('clf1',
                 MultiOutputClassifier(estimator=LinearSVC(C=1, max_iter=100000,
                                                           random_state=42)))])

In [25]:
y_pred_tuned = best_tuned_clf.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [26]:
# Training accuracy
print("The training accuracy is: ")
print(accuracy_score(y_train, best_tuned_clf.predict(X_train))*100)
print(" ")

# Test accuracy
print("The test accuracy is: ")
print(accuracy_score(y_test, y_pred_tuned)*100)
print(" ")

# Classification report
print("Classification report")
print(classification_report(y_test, y_pred_tuned, target_names=y.columns.values))

The training accuracy is: 
71.62038449801648
 
The test accuracy is: 
29.95117485505035
 
Classification report
                        precision    recall  f1-score   support

               related       0.86      0.91      0.89      5023
               request       0.77      0.63      0.69      1118
                 offer       0.00      0.00      0.00        26
           aid_related       0.74      0.70      0.72      2752
          medical_help       0.58      0.28      0.38       505
      medical_products       0.62      0.25      0.36       346
     search_and_rescue       0.71      0.22      0.33       179
              security       0.25      0.01      0.02       112
              military       0.56      0.28      0.37       218
                 water       0.78      0.62      0.69       463
                  food       0.82      0.72      0.77       749
               shelter       0.78      0.55      0.64       591
              clothing       0.76      0.30      0.43  

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [27]:
# Use pipeline2 which includes StartingVerbEstimator
pipeline2.fit(X_train, y_train)

y_pred_2 = pipeline2.predict(X_test)

# Training accuracy
print("The training accuracy is: ")
print(accuracy_score(y_train, pipeline2.predict(X_train))*100)
print(" ")

# Test accuracy
print("The test accuracy is: ")
print(accuracy_score(y_test, y_pred_2)*100)
print(" ")

# Classification report
print("Classification report")
print(classification_report(y_test, y_pred_2, target_names=y.columns.values))

The training accuracy is: 
71.68141592920354
 
The test accuracy is: 
29.98169057064388
 
Classification report
                        precision    recall  f1-score   support

               related       0.86      0.91      0.89      5023
               request       0.77      0.63      0.69      1118
                 offer       0.00      0.00      0.00        26
           aid_related       0.74      0.70      0.72      2752
          medical_help       0.58      0.28      0.38       505
      medical_products       0.62      0.25      0.36       346
     search_and_rescue       0.70      0.21      0.33       179
              security       0.40      0.02      0.03       112
              military       0.56      0.28      0.37       218
                 water       0.77      0.62      0.69       463
                  food       0.82      0.72      0.77       749
               shelter       0.78      0.55      0.65       591
              clothing       0.75      0.31      0.43  

In [28]:
# Use pipeline3 which includes StartingVerbEstimator
pipeline3.fit(X_train, y_train)

y_pred_3 = pipeline3.predict(X_test)

# Training accuracy
print("The training accuracy is: ")
print(accuracy_score(y_train, pipeline3.predict(X_train))*100)
print(" ")

# Test accuracy
print("The test accuracy is: ")
print(accuracy_score(y_test, y_pred_3)*100)
print(" ")

# Classification report
print("Classification report")
print(classification_report(y_test, y_pred_3, target_names=y.columns.values))

The training accuracy is: 
99.76604618044959
 
The test accuracy is: 
25.05340250228868
 
Classification report
                        precision    recall  f1-score   support

               related       0.81      0.97      0.89      5023
               request       0.88      0.43      0.58      1118
                 offer       0.00      0.00      0.00        26
           aid_related       0.79      0.61      0.69      2752
          medical_help       0.60      0.04      0.07       505
      medical_products       0.86      0.05      0.10       346
     search_and_rescue       0.62      0.03      0.05       179
              security       1.00      0.02      0.04       112
              military       1.00      0.02      0.04       218
                 water       0.96      0.20      0.33       463
                  food       0.86      0.48      0.62       749
               shelter       0.87      0.20      0.33       591
              clothing       0.60      0.06      0.11  

### 9. Export your model as a pickle file

In [29]:
saved_model = pickle.dumps(pipeline2)

In [30]:
clf_from_pickle = pickle.loads(saved_model)

In [31]:
clf_from_pickle

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x2844f9790>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('starting_verb_transformer',
                                                 StartingVerbExtractor())])),
                ('clf2',
                 MultiOutputClassifier(estimator=LinearSVC(max_iter=100000,
                                                           random_state=42)))])

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.